In [ ]:
import os
import subprocess
import sys

# 1. 設定基礎路徑
HOME = os.getcwd()
os.makedirs(os.path.join(HOME, "data"), exist_ok=True)
os.makedirs(os.path.join(HOME, "outputs"), exist_ok=True)
os.makedirs(os.path.join(HOME, "weights"), exist_ok=True)

# 2. 針對 P100 GPU 設定架構
os.environ["TORCH_CUDA_ARCH_LIST"] = "6.0"
os.environ["CUDA_HOME"] = "/usr/local/cuda"

print("🧹 步驟 1: 強制清理與安裝相容依賴...")

# 3. 移除衝突庫 (徹底清除)
pkgs_to_remove = ["numpy", "scipy", "torch", "torchvision", "torchaudio", "Pillow", "transformers", "groundingdino", "opencv-python", "pycocotools"]
subprocess.run(["pip", "uninstall", "-y"] + pkgs_to_remove)
subprocess.run(["rm", "-rf", "GroundingDINO"])

# 4. 安裝 PyTorch 2.1.0 (CUDA 12.1)
print("🔧 安裝 PyTorch 2.1.0...")
subprocess.run([
    "pip", "install", 
    "torch==2.1.0", "torchvision==0.16.0", "torchaudio==2.1.0", 
    "--index-url", "https://download.pytorch.org/whl/cu121"
])

# 5. 安裝其他依賴 (關鍵：Pillow 9.5.0)
print("🔧 安裝 Numpy < 2.0 與 Pillow 9.5.0...")
subprocess.run([
    "pip", "install", 
    "numpy<2.0", "scipy==1.13.1", "transformers==4.30.2", "opencv-python", "pycocotools",
    "--ignore-installed"
])

# ★★★ 強制鎖定 Pillow 9.5.0，解決 is_directory 錯誤 ★★★
subprocess.run(["pip", "install", "Pillow==9.5.0", "matplotlib==3.7.2", "--force-reinstall"])

# 6. 安裝 SAM 與 Supervision
subprocess.run(["pip", "install", "git+https://github.com/facebookresearch/segment-anything.git"])
subprocess.run(["pip", "install", "supervision==0.6.0"])

print("✅ 環境依賴安裝完成！請繼續執行 Cell 2。")

In [ ]:
import os

print("📥 步驟 2: 下載並編譯 GroundingDINO (這需要約 3-5 分鐘)...")

# 1. Clone 專案
if os.path.exists("GroundingDINO"):
    !rm -rf GroundingDINO
!git clone https://github.com/IDEA-Research/GroundingDINO.git

# 2. 強制設定環境變數
os.environ["CUDA_HOME"] = "/usr/local/cuda"
os.environ["BUILD_WITH_CUDA"] = "1"
os.environ["AM_I_DOCKER"] = "1"
os.environ["TORCH_CUDA_ARCH_LIST"] = "6.0" 

# 3. 開始編譯
%cd GroundingDINO
!python setup.py install
%cd {HOME}

print("✅ GroundingDINO 編譯完成！請執行下一個 Cell。")

In [ ]:
import os

HOME = os.getcwd()
WEIGHTS_DIR = os.path.join(HOME, "weights")
os.makedirs(WEIGHTS_DIR, exist_ok=True)

print("⬇️ 步驟 3: 下載模型權重...")

if not os.path.exists(os.path.join(WEIGHTS_DIR, "groundingdino_swint_ogc.pth")):
    !wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -P {WEIGHTS_DIR}

if not os.path.exists(os.path.join(WEIGHTS_DIR, "sam_vit_h_4b8939.pth")):
    !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {WEIGHTS_DIR}

print("✅ 權重下載完成！")

In [ ]:
import subprocess
import os

print("🚑 緊急修復環境中... (正在強制降級 NumPy 到 1.26.4)")

# 1. 強制移除當前的 numpy (無論它是什麼版本)
subprocess.run(["pip", "uninstall", "-y", "numpy"])

# 2. 強制安裝指定的舊版本
# 使用 --no-deps 參數，叫 pip 閉嘴，不要檢查相容性，直接裝就對了
subprocess.run(["pip", "install", "numpy==1.26.4", "--no-deps", "--force-reinstall"])

# 3. 確保 Matplotlib 也是舊版
subprocess.run(["pip", "install", "matplotlib==3.7.2", "--no-deps", "--force-reinstall"])

print("✅ 修復指令執行完畢！")
print("❗ 重要：請務必執行下一步『重啟 Session』，否則記憶體中的舊版 NumPy 不會消失。")

In [ ]:
import torch
import sys
import os

# 強制確保 Python 找得到 GroundingDINO
cwd = os.getcwd()
if os.path.join(cwd, "GroundingDINO") not in sys.path:
    sys.path.append(os.path.join(cwd, "GroundingDINO"))

print("🧠 步驟 4: 嘗試載入模型...")

try:
    from groundingdino.util.inference import load_model, load_image, predict
    from segment_anything import SamPredictor, sam_model_registry
    print("   ✅ 成功匯入函式庫！")
except ImportError as e:
    print(f"❌ 匯入失敗: {e}")
    raise e

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"   使用運算裝置: {DEVICE}")

# 1. 載入 GroundingDINO
HOME = os.getcwd()
WEIGHTS_DIR = os.path.join(HOME, "weights")
GD_CONFIG = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
GD_CHECKPOINT = os.path.join(WEIGHTS_DIR, "groundingdino_swint_ogc.pth")

grounding_dino_model = load_model(GD_CONFIG, GD_CHECKPOINT)

# 2. 載入 SAM
SAM_CHECKPOINT = os.path.join(WEIGHTS_DIR, "sam_vit_h_4b8939.pth")
sam = sam_model_registry["vit_h"](checkpoint=SAM_CHECKPOINT).to(DEVICE)
sam_predictor = SamPredictor(sam)

print("\n🎉 恭喜！模型載入成功！")

In [ ]:
import cv2
import numpy as np
import supervision as sv
import matplotlib.pyplot as plt
import torch
# 關鍵修正：從 GroundingDINO 匯入座標轉換工具
from groundingdino.util.box_ops import box_cxcywh_to_xyxy

def run_grounded_sam(image_path, text_prompt, box_threshold=0.35, text_threshold=0.25):
    image_source, image = load_image(image_path)
    
    # 1. GroundingDINO: Text -> Box
    boxes, logits, phrases = predict(
        model=grounding_dino_model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )
    
    if len(boxes) == 0:
        return image_source, None, None, None

    # 2. 座標轉換 (修正點)
    h, w, _ = image_source.shape
    boxes_xyxy = boxes * torch.Tensor([w, h, w, h])
    boxes_xyxy = box_cxcywh_to_xyxy(boxes_xyxy).numpy()
    
    # 3. SAM: Box -> Mask
    sam_predictor.set_image(image_source)
    transformed_boxes = sam_predictor.transform.apply_boxes_torch(
        torch.as_tensor(boxes_xyxy, device=DEVICE), 
        image_source.shape[:2]
    )
    
    masks, _, _ = sam_predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False,
    )
    return image_source, boxes_xyxy, masks, phrases

def plot_result(image, boxes, masks, phrases):
    box_annotator = sv.BoxAnnotator()
    mask_annotator = sv.MaskAnnotator()
    detections = sv.Detections(
        xyxy=boxes, 
        mask=masks.cpu().numpy().squeeze(1), 
        class_id=np.arange(len(boxes))
    )
    labels = [f"{phrase}" for phrase in phrases]
    annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)
    annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
    
    # 顯示圖片 (可選，批量跑時可以註解掉以免太佔版面)
    # plt.figure(figsize=(10, 10))
    # plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    
    return annotated_image

print("✅ Pipeline 定義完成 (修正版)！")

In [ ]:
import os
import json
import shutil

# --- 設定您的資料集路徑 (請修改這裡！) ---
# 假設您上傳的資料集名稱是 "my-custom-dataset"
# 路徑通常長這樣: "/kaggle/input/my-custom-dataset"
INPUTimg_ROOT = "/kaggle/input/redataset/val2017_50"
INPUTjson_ROOT = "/kaggle/input/redataset"

# 設定您的圖片資料夾名稱與 JSON 檔名
USER_IMAGE_DIR = os.path.join(INPUTimg_ROOT, "val2017_50")  # 如果圖片直接在根目錄，就改成 INPUT_ROOT
USER_JSON_PATH = os.path.join(INPUTjson_ROOT, "val2017_prompt.json")

# 設定輸出工作目錄
HOME = os.getcwd()
OUTPUT_DIR = os.path.join(HOME, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📂 準備讀取使用者資料集...")
print(f"   圖片來源: {USER_IMAGE_DIR}")
print(f"   Prompt設定檔: {USER_JSON_PATH}")

# 檢查檔案是否存在
if not os.path.exists(USER_JSON_PATH):
    print(f"❌ 錯誤：找不到 {USER_JSON_PATH}，請檢查路徑或是檔名是否正確。")
else:
    with open(USER_JSON_PATH, 'r') as f:
        data = json.load(f)
    print(f"✅ 成功讀取 JSON！共包含 {len(data)} 筆資料。")
    print(f"   第一筆範例: {data[0]}")
    
    # 檢查第一張圖是否存在
    first_img_path = os.path.join(USER_IMAGE_DIR, data[0]['image_filename'])
    if os.path.exists(first_img_path):
        print(f"✅ 圖片路徑檢查通過: {first_img_path}")
        print("請繼續執行 Cell 7 進行推論！")
    else:
        print(f"❌ 錯誤：找不到第一張圖片 {first_img_path}")
        print("   請確認 JSON 中的 file_name 是否與資料夾內的實際檔名一致。")

In [ ]:
import os
import cv2
import json
import torch
import numpy as np
import supervision as sv
import matplotlib.pyplot as plt
import zipfile
import shutil  # <--- 新增這個
from groundingdino.util.box_ops import box_cxcywh_to_xyxy

# --- 1. 設定路徑 ---
INPUTimg_ROOT = "/kaggle/input/redataset/val2017_50"
INPUTjson_ROOT = "/kaggle/input/redataset"
USER_JSON_PATH = os.path.join(INPUTjson_ROOT, "val2017_prompt.json")
USER_IMAGE_DIR = os.path.join(INPUTimg_ROOT, "val2017_50")
HOME = os.getcwd()
OUTPUT_DIR = os.path.join(HOME, "outputs")

# ★★★ 關鍵修正：執行前先清空輸出資料夾 ★★★
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR) # 刪除整個資料夾
os.makedirs(OUTPUT_DIR, exist_ok=True) # 重新建立空的
print(f"🧹 已清空 {OUTPUT_DIR}，準備存放新結果...")
# ------------------------------------------

# PROMPT_TYPE = 'coarse'
# PROMPT_TYPE = 'attribute'
PROMPT_TYPE = 'fine_grained'

print(f"🚀 步驟 7: 開始批量推論與產圖...")
# ... (以下程式碼保持不變) ...

# 檢查路徑
if not os.path.exists(USER_JSON_PATH):
    print(f"❌ 錯誤：找不到 JSON 檔 {USER_JSON_PATH}")
elif not os.path.exists(USER_IMAGE_DIR):
    if os.path.exists(INPUTimg_ROOT):
        USER_IMAGE_DIR = INPUTimg_ROOT
        print(f"⚠️ 路徑修正：改為使用 {USER_IMAGE_DIR}")
    else:
        print(f"❌ 錯誤：找不到圖片資料夾 {USER_IMAGE_DIR}")

if os.path.exists(USER_JSON_PATH) and os.path.exists(USER_IMAGE_DIR):
    with open(USER_JSON_PATH, 'r', encoding='utf-8') as f:
        image_data_list = json.load(f)

    success_count = 0

    for idx, item in enumerate(image_data_list):
        file_name = item['image_filename']
        try:
            prompt = item['prompts'][PROMPT_TYPE]
        except KeyError:
            continue

        img_path = os.path.join(USER_IMAGE_DIR, file_name)
        
        print(f"[{idx+1}/{len(image_data_list)}] 處理: {file_name} ...", end=" ")
        
        if not os.path.exists(img_path):
            print("❌ 找不到圖片")
            continue

        try:
            # A. Pipeline
            image_source, boxes, masks, phrases = run_grounded_sam(
                image_path=img_path, 
                text_prompt=prompt, 
                box_threshold=0.35, 
                text_threshold=0.25
            )
            
            # B. 繪圖存檔
            if boxes is not None and len(boxes) > 0:
                res = plot_result(image_source, boxes, masks, phrases)
                save_name = f"result_{os.path.splitext(file_name)[0]}.png"
                cv2.imwrite(os.path.join(OUTPUT_DIR, save_name), res)
                print("✅ 完成")
                success_count += 1
            else:
                print("⚠️ 未偵測到物件")
                
        except Exception as e:
            print(f"❌ 失敗: {e}")

    # 打包
    print("\n" + "="*40)
    print(f"📊 處理完成！成功產出 {success_count} 張圖片")

    # zip_name = "coarse_results.zip"
    # zip_name = "attribute_results.zip"
    zip_name = "fine_grained_results.zip" # 改個名字區分
    print(f"📦 正在打包成 {zip_name} ...")
    
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_name in os.listdir(OUTPUT_DIR):
            if file_name.endswith(".png"):
                full_path = os.path.join(OUTPUT_DIR, file_name)
                zf.write(full_path, arcname=file_name)
                
    print(f"✨ 全部完成！請下載 '{zip_name}' (已無雜檔)")

In [ ]:
import os
import sys
import subprocess
import json
import numpy as np
import torch
import cv2
from pycocotools.coco import COCO

# --- 0. 自動修復 Protobuf ---
try:
    import google.protobuf
    if google.protobuf.__version__ != '3.20.3':
        subprocess.check_call([sys.executable, "-m", "pip", "install", "protobuf==3.20.3"])
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "protobuf==3.20.3"])

# --- 1. 補上必要的匯入與函式定義 (解決 NameError) ---
try:
    from groundingdino.util.inference import load_image, predict, annotate
    from groundingdino.util.box_ops import box_cxcywh_to_xyxy
except ImportError:
    print("⚠️ 嚴重警告：無法匯入 GroundingDINO。請確認您是否已執行安裝步驟？")

# ★★★ 這裡補上了您缺失的 run_grounded_sam 函式 ★★★
def run_grounded_sam(image_path, text_prompt, box_threshold, text_threshold):
    # 檢查模型是否已載入
    if 'grounding_dino_model' not in globals() or 'sam_predictor' not in globals():
        raise NameError("模型未載入！請先執行之前的 Cell (Cell 4) 來載入 GroundingDINO 和 SAM 模型。")

    image_source, image = load_image(image_path)
    
    # 1. GroundingDINO 推論
    boxes, logits, phrases = predict(
        model=grounding_dino_model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )
    
    # 如果沒偵測到東西
    if len(boxes) == 0:
        return image_source, None, None, None

    # 2. 座標轉換 (Box -> SAM Ready)
    h, w, _ = image_source.shape
    boxes_xyxy = boxes * torch.Tensor([w, h, w, h])
    boxes_xyxy = box_cxcywh_to_xyxy(boxes_xyxy).numpy()
    
    # 3. SAM 推論
    sam_predictor.set_image(image_source)
    transformed_boxes = sam_predictor.transform.apply_boxes_torch(
        torch.as_tensor(boxes_xyxy, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')), 
        image_source.shape[:2]
    )
    
    masks, _, _ = sam_predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False,
    )
    return image_source, boxes_xyxy, masks, phrases

# ==========================================
# 2. 設定路徑 (Val2017)
# ==========================================
INPUTimg_ROOT = "/kaggle/input/redataset/val2017_50"
INPUTjson_ROOT = "/kaggle/input/redataset"
USER_JSON_PATH = os.path.join(INPUTjson_ROOT, "val2017_prompt.json")

# 路徑防呆
POSSIBLE_SUBDIR = os.path.join(INPUTimg_ROOT, "val2017_50")
if os.path.exists(POSSIBLE_SUBDIR):
    USER_IMAGE_DIR = POSSIBLE_SUBDIR
    print(f"✅ 自動修正圖片路徑: {USER_IMAGE_DIR}")
else:
    USER_IMAGE_DIR = INPUTimg_ROOT
    print(f"✅ 圖片路徑: {USER_IMAGE_DIR}")

# ==========================================
# 3. 準備 Ground Truth
# ==========================================
HOME = os.getcwd()
DATA_DIR = os.path.join(HOME, "data")
ANN_FILE_VAL = os.path.join(DATA_DIR, "annotations", "instances_val2017.json")

# 自動補下載
if not os.path.exists(ANN_FILE_VAL):
    print("⬇️ 正在補下載 COCO 標註檔...")
    os.makedirs(os.path.join(DATA_DIR, "annotations"), exist_ok=True)
    os.system(f"wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O {DATA_DIR}/annotations.zip")
    os.system(f"unzip -q {DATA_DIR}/annotations.zip -d {DATA_DIR}")

# ==========================================
# 4. 執行評估
# ==========================================
if not os.path.exists(USER_JSON_PATH):
    print(f"❌ 錯誤：找不到 JSON 檔 {USER_JSON_PATH}")
elif 'grounding_dino_model' not in globals():
    print("❌ 錯誤：模型變數不存在。請往回捲動，執行 **Cell 4 (載入模型)** 後再回來執行此 Cell。")
else:
    coco_api = COCO(ANN_FILE_VAL)
    
    with open(USER_JSON_PATH, 'r', encoding='utf-8') as f:
        image_data_list = json.load(f)

    print(f"📊 準備評估 {len(image_data_list)} 張圖片...")
    
    scores = {"iou": [], "precision": [], "recall": [], "dice": []}
    PROMPT_TYPE = 'attribute' 

    # 指標計算
    def calculate_metrics(pred_mask, gt_mask):
        pred = pred_mask > 0
        gt = gt_mask > 0
        intersection = np.logical_and(pred, gt).sum()
        union = np.logical_or(pred, gt).sum()
        tp = intersection
        fp = np.logical_and(pred, ~gt).sum()
        fn = np.logical_and(~pred, gt).sum()
        
        iou = intersection / union if union > 0 else 0.0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0.0
        return iou, precision, recall, dice

    valid_count = 0

    for idx, item in enumerate(image_data_list):
        file_name = item['image_filename']
        img_id = item.get('coco_original_id')
        
        try:
            prompt = item['prompts'][PROMPT_TYPE]
        except: continue 

        img_path = os.path.join(USER_IMAGE_DIR, file_name)
        # 檔名路徑修正
        if not os.path.exists(img_path):
            img_path = os.path.join(USER_IMAGE_DIR, os.path.basename(file_name))
            if not os.path.exists(img_path): continue

        # A. 取得 GT
        if img_id not in coco_api.imgs: continue
        ann_ids = coco_api.getAnnIds(imgIds=img_id)
        anns = coco_api.loadAnns(ann_ids)
        
        image_source, _ = load_image(img_path)
        h, w, _ = image_source.shape
        gt_mask_all = np.zeros((h, w), dtype=np.uint8)
        
        has_valid_gt = False
        for ann in anns:
            cat_name = coco_api.loadCats(ann['category_id'])[0]['name']
            is_match = (cat_name.lower() in prompt.lower()) or \
                       (prompt.lower().find(cat_name.lower()) != -1) or \
                       ("public transit bus" in prompt.lower() and "bus" in cat_name.lower())

            if is_match:
                mask = coco_api.annToMask(ann)
                gt_mask_all = np.maximum(gt_mask_all, mask)
                has_valid_gt = True
        
        if not has_valid_gt: continue
        valid_count += 1

        # B. 推論
        try:
            # 這裡呼叫剛剛補上的函式
            _, _, masks, _ = run_grounded_sam(
                image_path=img_path, text_prompt=prompt, 
                box_threshold=0.35, text_threshold=0.25
            )
            pred_mask_all = np.zeros((h, w), dtype=np.uint8)
            if masks is not None and len(masks) > 0:
                for m in masks.cpu().numpy().squeeze(1):
                    pred_mask_all = np.maximum(pred_mask_all, m.astype(np.uint8))
        except Exception as e:
            print(f"❌ 推論錯誤: {e}")
            pred_mask_all = np.zeros((h, w), dtype=np.uint8)

        # C. 算分
        iou, prec, rec, dice = calculate_metrics(pred_mask_all, gt_mask_all)
        scores['iou'].append(iou)
        scores['precision'].append(prec)
        scores['recall'].append(rec)
        scores['dice'].append(dice)
        
        if valid_count % 10 == 0:
            print(f"  [進度 {idx+1}] Valid IoU: {iou:.3f} | Dice: {dice:.3f}")

    # 輸出
    if len(scores['iou']) > 0:
        print("\n" + "="*40)
        print(f"🏆 最終評估報告 (使用 {PROMPT_TYPE} 提示)")
        print(f"   有效樣本數: {len(scores['iou'])} / {len(image_data_list)}")
        print("-" * 40)
        print(f"1. Mean IoU (重疊度):       {np.mean(scores['iou']):.4f}")
        print(f"2. Mean Precision (準確率): {np.mean(scores['precision']):.4f}")
        print(f"3. Mean Recall (召回率):    {np.mean(scores['recall']):.4f}")
        print(f"4. Mean Dice (F1-Score):   {np.mean(scores['dice']):.4f}")
        print("="*40)
    else:
        print("❌ 依然沒有有效樣本。")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# 設定存檔路徑
HOME = os.getcwd()
OUTPUT_DIR = os.path.join(HOME, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("🚀 步驟 9: 正在生成視覺化圖表...")

# 檢查是否有分數數據
if 'scores' not in globals() or len(scores['iou']) == 0:
    print("❌ 錯誤：找不到分數數據。請先成功執行 Cell 8 計算出 IoU。")
else:
    # --- 1. 繪製平均分數長條圖 (Bar Chart) ---
    metrics_name = ['IoU', 'Precision', 'Recall', 'Dice']
    metrics_mean = [
        np.mean(scores['iou']),
        np.mean(scores['precision']),
        np.mean(scores['recall']),
        np.mean(scores['dice'])
    ]
    colors = ['#4c72b0', '#55a868', '#c44e52', '#8172b3'] # 專業配色

    plt.figure(figsize=(10, 6))
    bars = plt.bar(metrics_name, metrics_mean, color=colors, alpha=0.85, width=0.6)

    # 標上數值
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{height:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.ylim(0, 1.0) # Y軸固定 0~1
    plt.title('Grounded-SAM Performance Metrics (Mean)', fontsize=15)
    plt.ylabel('Score', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    
    # 存檔
    bar_path = os.path.join(OUTPUT_DIR, "metrics_bar_chart.png")
    plt.savefig(bar_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"📊 長條圖已儲存: {bar_path}")

    # --- 2. 繪製分佈箱型圖 (Box Plot) ---
    # 這張圖能顯示模型穩定性
    data_to_plot = [scores['iou'], scores['precision'], scores['recall'], scores['dice']]

    plt.figure(figsize=(10, 6))
    box = plt.boxplot(data_to_plot, labels=metrics_name, patch_artist=True, 
                      medianprops=dict(color="black", linewidth=1.5))

    # 幫箱子上色
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)

    plt.title('Distribution of Evaluation Metrics (N=46)', fontsize=15)
    plt.ylabel('Score', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    
    # 存檔
    box_path = os.path.join(OUTPUT_DIR, "metrics_boxplot.png")
    plt.savefig(box_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"📦 箱型圖已儲存: {box_path}")

In [ ]:
import os
import json
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from pycocotools.coco import COCO

# --- 1. 設定路徑 ---
INPUTimg_ROOT = "/kaggle/input/redataset/val2017_50"
INPUTjson_ROOT = "/kaggle/input/redataset"
USER_JSON_PATH = os.path.join(INPUTjson_ROOT, "val2017_prompt.json")

# 圖片路徑自動偵測
POSSIBLE_SUBDIR = os.path.join(INPUTimg_ROOT, "val2017_50")
USER_IMAGE_DIR = POSSIBLE_SUBDIR if os.path.exists(POSSIBLE_SUBDIR) else INPUTimg_ROOT

# Ground Truth 準備
HOME = os.getcwd()
ANN_FILE_VAL = os.path.join(HOME, "data", "annotations", "instances_val2017.json")
OUTPUT_DIR = os.path.join(HOME, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 確保 GroundingDINO 函式可用
try:
    from groundingdino.util.inference import load_image
except:
    print("⚠️ 請先執行前面的 Cell 以載入必要函式！")

# --- 2. 定義評估流程 ---
def evaluate_prompt_type(prompt_type, image_data, coco_api):
    print(f"\n🚀 正在評估 Prompt 類型: 【{prompt_type}】...")
    scores = {"iou": [], "precision": [], "recall": [], "dice": []}
    
    # 計算指標函式
    def calculate_metrics(pred_mask, gt_mask):
        pred = pred_mask > 0
        gt = gt_mask > 0
        intersection = np.logical_and(pred, gt).sum()
        union = np.logical_or(pred, gt).sum()
        tp = intersection
        fp = np.logical_and(pred, ~gt).sum()
        fn = np.logical_and(~pred, gt).sum()
        
        iou = intersection / union if union > 0 else 0.0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0.0
        return iou, precision, recall, dice

    valid_cnt = 0
    for item in image_data:
        file_name = item['image_filename']
        img_id = item.get('coco_original_id')
        
        # 取得對應粒度的 Prompt
        try:
            prompt = item['prompts'][prompt_type]
        except: continue

        img_path = os.path.join(USER_IMAGE_DIR, file_name)
        if not os.path.exists(img_path):
            img_path = os.path.join(USER_IMAGE_DIR, os.path.basename(file_name))
            if not os.path.exists(img_path): continue

        if img_id not in coco_api.imgs: continue
        
        # 取得 GT
        ann_ids = coco_api.getAnnIds(imgIds=img_id)
        anns = coco_api.loadAnns(ann_ids)
        image_source, _ = load_image(img_path)
        h, w, _ = image_source.shape
        gt_mask_all = np.zeros((h, w), dtype=np.uint8)
        
        has_gt = False
        for ann in anns:
            cat_name = coco_api.loadCats(ann['category_id'])[0]['name']
            # 寬鬆比對
            if cat_name.lower() in prompt.lower() or prompt.lower().find(cat_name.lower()) != -1 or ("bus" in cat_name and "bus" in prompt):
                mask = coco_api.annToMask(ann)
                gt_mask_all = np.maximum(gt_mask_all, mask)
                has_gt = True
        
        if not has_gt: continue
        valid_cnt += 1

        # 推論
        try:
            _, _, masks, _ = run_grounded_sam(img_path, prompt, 0.35, 0.25)
            pred_mask_all = np.zeros((h, w), dtype=np.uint8)
            if masks is not None and len(masks) > 0:
                for m in masks.cpu().numpy().squeeze(1):
                    pred_mask_all = np.maximum(pred_mask_all, m.astype(np.uint8))
        except:
            pred_mask_all = np.zeros((h, w), dtype=np.uint8)

        # 算分
        iou, prec, rec, dice = calculate_metrics(pred_mask_all, gt_mask_all)
        scores['iou'].append(iou)
        scores['precision'].append(prec)
        scores['recall'].append(rec)
        scores['dice'].append(dice)

    # 回傳平均分
    return {
        "Mean IoU": np.mean(scores['iou']),
        "Mean Precision": np.mean(scores['precision']),
        "Mean Recall": np.mean(scores['recall']),
        "Mean Dice": np.mean(scores['dice'])
    }

# --- 3. 執行主程式 ---
if not os.path.exists(ANN_FILE_VAL):
    print("❌ 請先下載 COCO 標註檔")
else:
    coco_api = COCO(ANN_FILE_VAL)
    with open(USER_JSON_PATH, 'r') as f:
        data_list = json.load(f)

    # 定義要比較的粒度
    prompt_types = ['coarse', 'attribute', 'fine_grained']
    final_results = {}

    for p_type in prompt_types:
        final_results[p_type] = evaluate_prompt_type(p_type, data_list, coco_api)

    # --- 4. 生成圖表與報告 ---
    print("\n" + "="*50)
    print("🏆 不同粒度 Prompt 效能比較報告")
    print("="*50)
    
    # 轉成 DataFrame 顯示表格
    df = pd.DataFrame(final_results).T
    print(df)
    
    # 繪製長條比較圖
    df.plot(kind='bar', figsize=(10, 6), width=0.8, colormap='viridis')
    plt.title("Performance Comparison by Prompt Granularity", fontsize=15)
    plt.ylabel("Score (0-1)", fontsize=12)
    plt.xticks(rotation=0, fontsize=11)
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.legend(loc='lower right')
    
    save_path = os.path.join(OUTPUT_DIR, "prompt_granularity_comparison.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ 比較圖表已儲存至: {save_path}")
    print("👉 這張圖可以用來回答您關於「語言提示形式影響」的研究問題！")